<h1 align="center">Rich's Week 3 Project Notebook - Part 1</h1>
<h2 align="center">Coursera IBM Data Science Professional Certificate</h2>
This notebook will primarily be used to fulfill the requirements of the Coursera IBM Data Science Capstone project.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h2>Rich Huebner</h2>
<h3>Week 3 Project Assignment - Neighborhoods in Toronto</h3>

In [41]:
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import geopy
import requests
from urllib.request import urlopen
from sklearn.cluster import KMeans
from IPython.display import display_html

print('Libraries are now included.')

Libraries are now included.


In [52]:
# Calls to the Foursquare API
# https://api.foursquare.com/v2/search/
# include ClientID, Client Secret, Version.
CLIENT_ID = '0PNU1Y1L1N0W33QOPPR0JMJYJRASC2NNJSCH5JPXXFEO2BIY' # your Foursquare ID
CLIENT_SECRET = '2LX3Z3HXAEKA1DZTT54TQDE4ZMSCYSLAZGKGZSK5ZIQ4IYMP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('    Client ID: ' + CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)

    Client ID: 0PNU1Y1L1N0W33QOPPR0JMJYJRASC2NNJSCH5JPXXFEO2BIY
Client Secret: 2LX3Z3HXAEKA1DZTT54TQDE4ZMSCYSLAZGKGZSK5ZIQ4IYMP


In [20]:
# Clean, straightforward way to grab the table using pandas read_html command.
toronto_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = pd.read_html(toronto_url)
df = toronto_data[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [35]:
# Create a new df where Borough != 'Not assigned'
df_na = df[df.Borough != 'Not assigned']
df_na = df_na.sort_values(by = ['Postal Code', 'Borough'])

# Remove space from the name of the attribute, so we can merge this later - basically renaming the columns.
df_na.columns = ['Postcode', 'Borough', 'Neighborhood']
df_na.head()

,Postcode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [30]:
df_na.shape

(103, 3)

<h2>Part II:  Geographic Coordinates - Lat/Long</h2>

In [33]:
# Going to yank these from a csv file.
geourl = "https://cocl.us/Geospatial_data"
geodata = pd.read_csv(geourl)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
# Going to rename the columns for ease of use
geodata.columns = ['Postcode', 'lat', 'lon']
# and merge the data on Postcode.
toronto_data = pd.merge(df_na, geodata, on = 'Postcode')

In [60]:
# Let's look at the combined dataset now. I like it!
# Also do a little more renaming for ease of analysis later on.
toronto_data.columns = ['postcode', 'borough', 'nbh', 'lat', 'lon']
toronto_data


,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


<h2>Part III:  Clustering and Mapping the Toronto Boroughs </h2>

In [61]:
# First, let's make sure we have Toronto on the map - of where the center of it is -- for folium map.
address = 'Toronto, Ontario'
geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical center of Toronto, Ontario Canada is:  {}, {}.'.format(latitude, longitude))


The geograpical center of Toronto, Ontario Canada is:  43.6534817, -79.3839347.


In [77]:
# Check to make sure we have a folium map of Toronto to begin with.
tmap = folium.Map(location = [latitude, longitude], zoom_start = 13)
tmap

In [63]:
td = toronto_data
td.head()

,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [76]:
# Next, for this part, let's fill in the various neighborhoods.

for lat, lng, nb in zip(td['lat'], td['lon'], td['nbh']):
    label = '{}'.format(nb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#112233',
        fill_opacity=0.2,
        parse_html=False).add_to(tmap)  
    
tmap

    

In [75]:
# Define limits of results and radius (in meters)
LIMIT = 10
radius = 100
for n, lat, long in zip(td.nbh, td.lat, td.lon):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json
    
tmap
    